In [7]:
import os
import glob
import random
import pandas as pd
import tiktoken

# 사용자 데이터프레임 로드 (예시 파일 경로 사용)
# tsv 파일을 DataFrame으로 읽기
user = pd.read_csv('user.tsv', sep='\t', names=['User', 'History', 'Train', 'Test'])
user_raw = pd.read_csv('user(raw).tsv', sep='\t',  names=['User', 'History', 'Train', 'Test'])

history_news = pd.read_csv('history/news.tsv', sep='\t',  names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])

train_news = pd.read_csv('train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
train_behaviors = pd.read_csv('train/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])

test_news = pd.read_csv('test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
test_behaviors = pd.read_csv('test/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])

# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

# publish 순서에 맞게 오름차순으로 정렬
train_news_sorted = train_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

# With Negative Prompt 생성 및 token 계산

In [10]:
# 사용자별 input과 output 데이터를 저장할 딕셔너리
user_data = {}
user_metadata = []
hidden_positions_data = []

# 사용할 gpt (gpt별 사용할 token 계산용도)
gpt_name = "gpt-3.5-turbo"
# gpt_name = "gpt-4o-mini"

# 사용자 ID 범위 설정
user_count = 1000
user_ids = [f'U{i}' for i in range(1, user_count + 1)]

for ID in user_ids:
    history = user[user['User'] == ID]['History'].iloc[0]

    # History에서 News ID 추출
    news_ids = []
    [news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

    # News ID의 맞는 Title 추출
    titles = []
    for news_id in news_ids:
        matching_rows = history_news[history_news['News ID'] == news_id]
        if not matching_rows.empty:
            titles.append(matching_rows.iloc[0]['Title'])

    train = user[user['User'] == ID]['Train'].iloc[0]

    # Train에서 News ID 추출
    train_ids = []
    [train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

    # News ID의 맞는 Title 추출
    train_titles = []
    for train_id in train_ids:
        matching_rows = train_news[train_news['News ID'] == train_id]
        if not matching_rows.empty:
            train_titles.append(matching_rows.iloc[0]['Title'])

    # news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
    negative_ids = []
    negative_titles = []
    used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

    for news_id in news_ids:
        idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
        above_idx, below_idx = idx - 1, idx + 1
        similar_ids = []
        similar_titles = []

        # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 두 개의 뉴스를 찾기
        above_count = 0
        while above_idx >= 0 and above_count < 2:
            if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
                similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
                similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
                used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
                above_count += 1
            above_idx -= 1

        # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 두 개의 뉴스를 찾기
        below_count = 0
        while below_idx < len(history_news_sorted) and below_count < 2:
            if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
                similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
                similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
                used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
                below_count += 1
            below_idx += 1

        negative_ids.append(similar_ids)
        negative_titles.append(similar_titles)

    # Train 데이터에 대한 유사한 뉴스 기사 찾기
    negative_train_ids = []
    negative_train_titles = []
    used_ids = set(train_ids)  # 중복을 방지하기 위해 used_ids에 train_id를 추가

    for train_id in train_ids:
        idx = train_news_sorted[train_news_sorted['News ID'] == train_id].index[0]
        above_idx, below_idx = idx - 1, idx + 1
        similar_ids = []
        similar_titles = []

        # 이미 존재하는 News ID와 중복된 News를 제외하고 위에서 가장 가까운 두 개의 News를 찾기
        while above_idx >= 0 and len(similar_ids) < 2:
            if train_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
                similar_ids.append(train_news_sorted.loc[above_idx, 'News ID'])
                similar_titles.append(train_news_sorted.loc[above_idx, 'Title'])
                used_ids.add(train_news_sorted.loc[above_idx, 'News ID'])
            above_idx -= 1

        # 이미 존재하는 News ID와 중복된 News를 제외하고 아래에서 가장 가까운 두 개의 News를 찾기
        while below_idx < len(train_news_sorted) and len(similar_ids) < 4:
            if train_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
                similar_ids.append(train_news_sorted.loc[below_idx, 'News ID'])
                similar_titles.append(train_news_sorted.loc[below_idx, 'Title'])
                used_ids.add(train_news_sorted.loc[below_idx, 'News ID'])
            below_idx += 1

        negative_train_ids.append(similar_ids)
        negative_train_titles.append(similar_titles)

    question_ids = [negative_list + [title] for negative_list, title in zip(negative_train_ids, train_ids)]
    question_titles = [negative_list + [title] for negative_list, title in zip(negative_train_titles, train_titles)]

    hidden_positions = []

    # 각 행을 섞고 히든 값의 위치를 저장
    for row in question_titles:
        hidden_value = row[-1]  # 히든 값 (마지막 요소)
        random.shuffle(row)  # 전체 행을 섞기
        hidden_index = row.index(hidden_value)  # 히든 값의 새로운 위치 찾기
        hidden_positions.append(hidden_index + 1)  # 히든 값의 위치 저장

    hidden_positions_data.append((ID, hidden_positions))
    number = len(titles)

    # 결과 저장
    user_content = "[News of interest to users]\n"
    for i in range(number):
        combined_titles = [titles[i]] + negative_titles[i]
        random.shuffle(combined_titles)
        user_content += f"{i + 1}) " + " / ".join(combined_titles) + "\n"
        user_content += f"Of the five news above, the news that the user is most interested in : {titles[i]}\n\n"

    user_content += "[Questions]\nRank the five candidates for each question based on the user's news interests.\n"
    for i, titles in enumerate(question_titles):
        user_content += f"Question {i + 1}) "
        user_content += " / ".join([f"{j + 1}: {title}" for j, title in enumerate(titles)]) + "\n"

    user_content += "\nDo not explain reasons in the response, just return a list of numbers for each article.\n"

    user_data[ID] = user_content
    user_metadata.append((ID, number, len(question_titles)))

# 파일 경로 지정 (input 파일이 저장될 폴더 경로)
output_folder_path = "user_prompts/with_negative"
os.makedirs(output_folder_path, exist_ok=True)

# 각 사용자별로 텍스트 파일 생성
for user_id, content in user_data.items():
    file_path = os.path.join(output_folder_path, f"{user_id}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)

# 각 사용자별 토큰 수와 output 값을 저장할 리스트
token_counts_and_outputs = []

# tiktoken을 사용하여 토큰 수 계산
encoding = tiktoken.encoding_for_model(gpt_name)

# 모든 텍스트 파일에 대해 반복 수행
for file_path in sorted(glob.glob(os.path.join(output_folder_path, "*.txt"))):
    with open(file_path, "r", encoding="utf-8") as file:
        # 파일에서 input 텍스트 추출
        input_text = file.read()

        # 토큰 수 계산
        token_count = len(encoding.encode(input_text))

        # 사용자 ID 추출
        user_id = os.path.basename(file_path).split(".")[0]

        # Question 수에 따른 output count 계산
        num_questions = input_text.count("Question ")
        output_count = 18 + (num_questions - 1) * 19 if num_questions > 0 else 0

        # 사용자 메타데이터에서 number와 num_questions 가져오기
        number = next(item[1] for item in user_metadata if item[0] == user_id)

        # 사용자 ID, 토큰 수, output count, news_ids 수, question 수를 리스트에 추가
        token_counts_and_outputs.append((user_id, token_count, output_count, number, num_questions))

# # 결과 출력
# for user_id, token_count, output_count, num_news_ids, num_questions in sorted(token_counts_and_outputs, key=lambda x: int(x[0][1:])):
#     print(f"User ID: {user_id:<5} Token Count: {token_count:<6} Output Count: {output_count:<4} History 수: {num_news_ids:<3} Question 수: {num_questions}")


# 결과 출력 (간격 맞추기)
meta_folder_path = "user_prompts/with_negative/metadata"
os.makedirs(meta_folder_path, exist_ok=True)
meta_file_path = os.path.join(meta_folder_path, "output_metadata.txt")

total_input_tokens = 0
total_output_count = 0

with open(meta_file_path, "w", encoding="utf-8") as meta_file:
    for user_id, token_count, output_count, num_news_ids, num_questions in sorted(token_counts_and_outputs, key=lambda x: int(x[0][1:])):
        output_line = (f"User ID: {user_id:<5} Input Tokens: {token_count:<6} Output Tokens: {output_count:<4}  "
                       f"History 수: {num_news_ids:<3}  Question 수: {num_questions}")
        total_input_tokens += token_count
        total_output_count += output_count

        # print(output_line)
        meta_file.write(output_line + "\n")

    # 전체 토큰 수와 출력 결과 저장
    total_line = f"\nTotal Input Tokens: {total_input_tokens}\nTotal Output Tokens: {total_output_count}"
    print(total_line)
    meta_file.write(total_line + "\n")


# hidden_positions 저장
hidden_positions_file_path = os.path.join(meta_folder_path, "hidden_positions.txt")
with open(hidden_positions_file_path, "w", encoding="utf-8") as hidden_file:
    for user_id, positions in hidden_positions_data:
        hidden_file.write(f"{user_id:<5}: {positions}\n")


Total Input Tokens: 3192670
Total Output Tokens: 243359


### token수만 계산

In [9]:
def process_user_data(filename, start_user, end_user):
    input_tokens_sum = 0
    output_tokens_sum = 0
    user_data = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()
        start_found = False
        
        for line in lines:
            if start_user in line:
                start_found = True
            
            if start_found:
                user_data.append(line.strip())
                parts = line.split()
                input_tokens = int(parts[5])
                output_tokens = int(parts[8])
                input_tokens_sum += input_tokens
                output_tokens_sum += output_tokens
                
            if end_user in line:
                break

    # 결과 출력
    for data in user_data:
        print(data)
    print(f"\nInput Tokens 합: {input_tokens_sum}")
    print(f"Output Tokens 합: {output_tokens_sum}")

# 파일 경로 및 USER 범위 지정
filename = 'user_prompts/with_negative/metadata/output_metadata.txt'
start_user = 'U1'
end_user = 'U50'
process_user_data(filename, start_user, end_user)

User ID: U1    Input Tokens: 2288   Output Tokens: 75    History 수: 18   Question 수: 4
User ID: U2    Input Tokens: 2410   Output Tokens: 132   History 수: 16   Question 수: 7
User ID: U3    Input Tokens: 2115   Output Tokens: 132   History 수: 14   Question 수: 7
User ID: U4    Input Tokens: 1839   Output Tokens: 75    History 수: 14   Question 수: 4
User ID: U5    Input Tokens: 3134   Output Tokens: 265   History 수: 17   Question 수: 14
User ID: U6    Input Tokens: 3703   Output Tokens: 379   History 수: 17   Question 수: 20
User ID: U7    Input Tokens: 5801   Output Tokens: 474   History 수: 34   Question 수: 25
User ID: U8    Input Tokens: 1877   Output Tokens: 189   History 수: 9    Question 수: 10
User ID: U9    Input Tokens: 2409   Output Tokens: 75    History 수: 19   Question 수: 4
User ID: U10   Input Tokens: 2497   Output Tokens: 189   History 수: 15   Question 수: 10
User ID: U11   Input Tokens: 4604   Output Tokens: 379   History 수: 27   Question 수: 20
User ID: U12   Input Tokens: 3360   O